In [1]:
from matplotlib import pyplot as plt
import numpy as np
import random
import time

In [2]:
class Individual:
    def __init__(self, functions, lb, ub):
        self.functions = functions
        self.lb = lb
        self.ub = ub
        self.code = np.random.random(len(lb)) * (ub - lb) + lb
        self.fvalue = np.empty(len(functions))
        self.evaluate()

    def evaluate(self):
        for i in range(len(self.functions)):
            self.fvalue[i] = self.functions[i](self.code)

In [39]:
# Function to find Pareto front

def find_pareto_front(population):
    pop_size = len(population)
    fNum = len(population[0].functions)
    pareto_front = []
    pareto_front_ids = []
    
    for i in range(pop_size):
        is_pareto_optimal = True

        for j in range(pop_size):
            if i != j:
                is_dominated = True
                for k in range(fNum):
                    is_dominated &= population[i].fvalue[k] > population[j].fvalue[k]
                if is_dominated:
                    is_pareto_optimal = False
                    break
        if is_pareto_optimal:
            pareto_front.append(population[i])
            pareto_front_ids.append(i)

    return pareto_front_ids, pareto_front

In [40]:
# Tournament selection, picking a random member of the tournaments Pareto front

def selection(population, tournament_size):
    chosen = random.sample(population, tournament_size)

    pareto_front = find_pareto_front(chosen)

    return random.choice(pareto_front)

In [41]:
# Whole arithmetic recombination

def crossover(parent1, parent2, child1, child2, alpha = 0.5):
    for i in range(len(child1.code)):
        child1.code[i] = parent1.code[i] * alpha + parent2.code[i] * (1 - alpha)
        child2.code[i] = parent2.code[i] * alpha + parent1.code[i] * (1 - alpha)

In [42]:
# Uniform mutation

def mutation(individual, mutation_prob):
    for i in range(len(individual.code)):
        if np.random.random() < mutation_prob:
            individual.code[i] = np.random.random() * (individual.ub[i] - individual.lb[i]) + individual.lb[i]

In [103]:
# Non-dominated sorting algorithm

def non_dominated_sort(population, num_fronts):
    removable_population = np.array(population.copy())
    pareto_fronts = []

    for i in range(num_fronts):
        pareto_front_ids, pareto_front = find_pareto_front(removable_population)
        if len(pareto_front_ids) == 0:
            break
        pareto_fronts.append(pareto_front)
        removable_population = np.delete(removable_population, pareto_front_ids)
    return pareto_fronts

In [130]:
def nsga(functions, lb, ub, pop_size, num_generations, num_fronts, tournament_size, elitism_size, crossover_alpha, mutation_prob, showGraph = True):
    start_time = time.time()
    
    # Initializing population
    population = [Individual(functions, lb, ub) for _ in range(pop_size)]
    new_population = population.copy()

    fNum = len(population[0].functions)

    pareto_fronts = non_dominated_sort(population, num_fronts)

    print(pareto_fronts)

In [131]:
# Schaffer function N.1
# -A <= X <= A, where A is between 10 and 10e5, higher A's increasing the difficulty of the problem

def f1(x):
    return x[0]**2

def f2(x):
    return (x[0]-2)**2

A = 10
lower_bounds = np.array([-A])
upper_bounds = np.array([A])

nsga(functions=[f1,f2], lb=lower_bounds, ub=upper_bounds, pop_size=10, num_generations=100, num_fronts = 5, tournament_size=5, elitism_size=50, crossover_alpha=0.3, mutation_prob=0.05)

[[<__main__.Individual object at 0x7cb6f4f7b160>, <__main__.Individual object at 0x7cb6f5b416c0>], [<__main__.Individual object at 0x7cb6f4f7a890>, <__main__.Individual object at 0x7cb6f4f79510>], [<__main__.Individual object at 0x7cb6f4f7b880>], [<__main__.Individual object at 0x7cb6f4f7a2f0>], [<__main__.Individual object at 0x7cb6f4f79e10>]]
